In [1]:
# Importation des bases
import pandas as pd
import functions as fc

dict_data = fc.read_all_raw(fc.list_bases)
dict_data.keys()

dict_keys(['rejets', 'etablissements', 'emissions', 'Trait_dechets_non_dangereux', 'Trait_dechets_dangereux', 'Prod_dechets_non_dangereux', 'Prod_dechets_dangereux', 'Prelevements'])

In [2]:
df = fc.clean_data_etab(
    dict_data['etablissements']
    )

In [3]:
requete_type = '''{{ 
  "query": {{
    "bool": {{
      "should": [
        {{ "match": {{ "rs_denom":   "{nom_etablissement}" }}}}
      ],
      "filter": [
        {{ "match":  {{ "adr_et_post": "{code_postal}" }}}},
        {{ "prefix":  {{ "apet": "{code_apet}" }}}}
      ]
    }}
  }},
  "size": 1
}}'''




# Il est nécessaire de spécifier l'index associé à chaque requête
header = '{"index" : "sirus_2020"}'

multiple_requetes = ""

# On itère sur le dataframe d'établissements polluants pour ajouter une requête spécifique à chacun d'entre eux
for index, row in df.loc[:,['nom_etablissement',  'code_postal', 'code_apet']].iterrows():
    
    multiple_requetes+= header
    multiple_requetes+= '\n'
    multiple_requetes+= requete_type.format_map(row).replace("\n","")
    multiple_requetes+= '\n'



res = fc.es.msearch(body = multiple_requetes, max_concurrent_searches = 500)

temp = pd.json_normalize(res,record_path=["responses"])

out_elastic = temp.apply(
        lambda l: fc.get_product_echo(l['hits.hits']),
        axis=1, result_type="expand")
out_elastic.head(1)

/home/coder/local/bin/conda/envs/basesspcloud/lib/python3.10/site-packages/elasticsearch/connection/base.py:200: ElasticsearchWarning: Elasticsearch built-in security features are not enabled. Without authentication, your cluster could be accessible to anyone. See https://www.elastic.co/guide/en/elasticsearch/reference/7.17/security-minimal-setup.html to enable security.
  warnings.warn(message, category=ElasticsearchWarning)


,sirus_id,nic,ape,apet,eff_3112_et,eff_etp_et,eff_et_effet_daaaammjj,enseigne_et1,nom_comm_et,adr_et_loc_geo,...,eff_etp_unitelegale,eff_effet_daaaammjj_unitelegale,siret_id,siret,longitude,latitude,geo_score,geo_type,geo_adresse,location
0,213801707,00075,8411Z,3700Z,,,,SERVICE ASSAINISSEMENT,,38170,...,62.0,20181231,21380170700075,21380170700075,5.663968,45.257715,0.96,housenumber,2 Rue Fétola 38120 Fontanil-Cornillon,"45.257715, 5.663968"


In [ ]:
df_out = pd.concat(
    [df, out_elastic]
    )

## Les émissions de CO2

On va commencer par se focaliser sur les gros émetteurs de CO2

In [ ]:
emissions = dict_data['emissions']
emissions_co2 = emissions.loc[emissions['polluant'].str.contains("CO2")]
emissions_co2 = emissions_co2.loc[emissions_co2['polluant'].str.contains('total')]
emissions_co2.head(1)

In [ ]:
emissions_co2['quantite'].div(1000).plot.hist(logx = True)

In [ ]:
dict_data.head(1)

In [ ]:
dict_data['emissions']["annee_emission"].value_counts()

In [ ]:
dict_data['emissions']["milieu"].value_counts()

In [ ]:
dechets_dang = dict_data['Trait_dechets_dangereux']

In [ ]:
dict_data['etablissements']['identifiant'].nunique()

In [ ]:
dict_data['etablissements'].head(1)

In [ ]:
dict_data